In [15]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.datasets import mnist
import os


In [22]:
# Load MNIST dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Filter even digits: 0, 2, 4, 6, 8
even_digits = [0, 2, 4, 6, 8]
train_mask = np.isin(train_labels, even_digits)
test_mask = np.isin(test_labels, even_digits)

train_images = train_images[train_mask]
train_labels = train_labels[train_mask]
test_images = test_images[test_mask]
test_labels = test_labels[test_mask]

# Map labels: 0→0, 2→1, 4→2, 6→3, 8→4
label_mapping = {0: 0, 2: 1, 4: 2, 6: 3, 8: 4}
train_labels = np.vectorize(label_mapping.get)(train_labels)
test_labels = np.vectorize(label_mapping.get)(test_labels)

# Normalize and reshape
train_images = train_images.astype("float32") / 255.0
test_images = test_images.astype("float32") / 255.0
train_images = np.expand_dims(train_images, -1)
test_images = np.expand_dims(test_images, -1)

# Split training set: 80% train, 20% validation
val_split = int(0.8 * len(train_images))
x_train = train_images[:val_split]
y_train = train_labels[:val_split]
x_val = train_images[val_split:]
y_val = train_labels[val_split:]


In [19]:
def create_model():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(5, activation='softmax')  # 5 classes for even digits
    ])
    return model


In [23]:
# Create and compile model
model = create_model()
model.compile(optimizer=Adam(learning_rate=0.002),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Set model checkpoint to save best model
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True, mode='min', verbose=1)

# Train for first 10 epochs
model.fit(x_train, y_train,
          epochs=10,
          batch_size=16,
          validation_data=(x_val, y_val),
          callbacks=[checkpoint])


Epoch 1/10
1473/1475 [============================>.] - ETA: 0s - loss: 0.0698 - accuracy: 0.9779
Epoch 1: val_loss improved from inf to 0.07457, saving model to best_model.h5
1475/1475 [==============================] - 33s 22ms/step - loss: 0.0698 - accuracy: 0.9779 - val_loss: 0.0746 - val_accuracy: 0.9769
Epoch 2/10
1473/1475 [============================>.] - ETA: 0s - loss: 0.0229 - accuracy: 0.9931
Epoch 2: val_loss improved from 0.07457 to 0.02640, saving model to best_model.h5
1475/1475 [==============================] - 32s 22ms/step - loss: 0.0229 - accuracy: 0.9931 - val_loss: 0.0264 - val_accuracy: 0.9917
Epoch 3/10
1474/1475 [============================>.] - ETA: 0s - loss: 0.0164 - accuracy: 0.9948
Epoch 3: val_loss did not improve from 0.02640
1475/1475 [==============================] - 30s 21ms/step - loss: 0.0164 - accuracy: 0.9948 - val_loss: 0.0289 - val_accuracy: 0.9930
Epoch 4/10
1473/1475 [============================>.] - ETA: 0s - loss: 0.0109 - accuracy: 0.9

In [ ]:
# Freeze 1st and 2nd convolutional layers
#Freeze 1st & 2nd convolutional layers (they keep their learned patterns).

model.layers[0].trainable = False  # Conv2D layer 1
model.layers[1].trainable = False  # MaxPool layer 1
model.layers[2].trainable = False  # Conv2D layer 2
model.layers[3].trainable = False  # MaxPool layer 2

# Recompile the model after freezing
model.compile(optimizer=Adam(learning_rate=0.002),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
#This technique is used to preserve early features and improve generalization. Often used in transfer learning too.
# Train remaining layers for next 10 epochs
model.fit(x_train, y_train,
          epochs=10,
          batch_size=16,
          validation_data=(x_val, y_val),
          callbacks=[checkpoint])


Epoch 1/10
1473/1475 [============================>.] - ETA: 0s - loss: 0.0028 - accuracy: 0.9994
Epoch 1: val_loss did not improve from 0.02188
1475/1475 [==============================] - 15s 10ms/step - loss: 0.0028 - accuracy: 0.9994 - val_loss: 0.0477 - val_accuracy: 0.9949
Epoch 2/10
1468/1475 [============================>.] - ETA: 0s - loss: 0.0012 - accuracy: 0.9999
Epoch 2: val_loss did not improve from 0.02188
1475/1475 [==============================] - 14s 9ms/step - loss: 0.0012 - accuracy: 0.9999 - val_loss: 0.0372 - val_accuracy: 0.9963
Epoch 3/10
1474/1475 [============================>.] - ETA: 0s - loss: 4.9348e-05 - accuracy: 1.0000
Epoch 3: val_loss did not improve from 0.02188
1475/1475 [==============================] - 16s 11ms/step - loss: 4.9329e-05 - accuracy: 1.0000 - val_loss: 0.0435 - val_accuracy: 0.9964
Epoch 4/10
1472/1475 [============================>.] - ETA: 0s - loss: 1.2403e-05 - accuracy: 1.0000
Epoch 4: val_loss did not improve from 0.02188
1475

In [26]:
# Load the best saved model
best_model = tf.keras.models.load_model('best_model.h5')

# Evaluate on test data
test_loss, test_acc = best_model.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_acc:.4f}")
print(f"Test loss: {test_loss:.4f}")


154/154 [==============================] - 2s 10ms/step - loss: 0.0139 - accuracy: 0.9945
Test accuracy: 0.9945
Test loss: 0.0139
